# Introduction to Azure OpenAI Function Calling


## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling


## Time

You should expect to spend 2 minutes running this sample.


## About this example

The objective of the provided Python file is to learn about Azure OpenAI Function Calling using the Azure OpenAI API.


## Installation

Refer to the README.md file in this folder for installation instructions.


In [ ]:
# Install the packages
%pip install -r requirements.txt

## Parameters


In [ ]:
import os
import json
import sqlite3
import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(".env")

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

con = sqlite3.connect("./database/order_data.db")

instructions = None

In [ ]:
def get_revenue_by_region(region: str = None) -> pd.DataFrame:
    params = []
    query = """
        SELECT
            Region,
            SUM(Number_of_Orders) AS Number_of_Orders,
            CAST(SUM(Sum_of_Order_Value_USD) AS INTEGER) AS Total_Order_Value,
            SUM(Sum_of_Order_Value_USD) / SUM(Number_of_Orders) AS Avg_Order_Value,
            SUM(Sum_of_Shipping_Cost_USD) / SUM(Number_of_Orders) AS Avg_Shipping_Cost
        FROM order_data
    """

    if region:
        query += "WHERE UPPER(Region) = UPPER(?)\n"
        params.append(region)

    query += "GROUP BY Region ORDER BY Region"

    return pd.read_sql_query(query, con, params=params)

In [ ]:
def get_sales_by_month(month: int) -> pd.DataFrame:
    query = """
        SELECT   
            SUM(Sum_of_Order_Value_USD) AS Revenue, Month, Year
            FROM order_data WHERE Month = ? GROUP BY Month, Year ORDER BY Year, Month
    """

    return pd.read_sql_query(query, con, params=[month])

### Create an AzureOpenAI client


In [ ]:
client = AzureOpenAI(
    api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint
)

### Define the Assistant tools


In [ ]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_revenue_by_region",
            "description": "Get the sales revenue for Contoso by region.",
            "parameters": {
                "type": "object",
                "properties": {
                    "region": {"type": "string"},
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_sales_by_month",
            "description": "Get the Contoso sales by month.",
            "parameters": {
                "type": "object",
                "properties": {
                    "month": {
                        "type": "integer",
                        "description": "The Contoso sales for a specific month.",
                    }
                },
                "required": ["month"],
            },
        },
    },
]

### Process Function calling


In [ ]:
def call_functions(tool_calls) -> None:
    function_map = {
        "get_revenue_by_region": get_revenue_by_region,
        "get_sales_by_month": get_sales_by_month,
    }

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        print(f"\nFunction Name: {function_name}, Function Args: {function_args}\n")

        function = function_map.get(function_name)

        if function:
            if function_name == "get_revenue_by_region":
                result_df = function(region=function_args.get("region"))
            elif function_name == "get_sales_by_month":
                result_df = function(month=function_args.get("month"))

            display(result_df)

## Instructions for role system assignment


In [ ]:
instructions = "".join((
    "You are a sales analysis assistant for Contoso.",
    "Please be polite, professional, helpful, and friendly.",
    "You get all the sales data from this app using the functions provided.",
    "If a question is not related to sales or you cannot answer the question,",
    "say, 'contact IT for more assistance.'",
    "If the user asks for help or says 'help', provide a list of sample questions that you can answer."
))

## Construct the Assistant and send to Azure OpenAI


In [ ]:
def process_message(question: str):

    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": question}
    ]

    response = client.chat.completions.create(
        model=api_deployment_name,
        messages=messages,
        tools=tools_list,
    )

    response_message = response.choices[0].message
    tool_calls = getattr(response_message, 'tool_calls', [])

    if tool_calls:
        call_functions(tool_calls)
    else:
        display(f"Response message: {response_message.content}")
        return

In [ ]:
process_message("What is the revenue for the region of Africa?")

In [ ]:
process_message("What is the revenue for the region of north america?")

In [ ]:
process_message("What is the revenue for all regions?")

In [ ]:
process_message("What were the sales for April?")

In [ ]:
process_message("What is the meaning of life?")